<a href="https://colab.research.google.com/github/anabellatus/machine_learning/blob/main/ML_Tugas2_JS4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tugas Jobsheet 4 Machine learning
### Nama : Ana Bellatus Mustaqfiro
### NIM : 2241720095
### Kelas : TI-3F / 04

**Tugas 2**

Buatlah model klasifikasi dengan menggunakan SVM berdasarkan data pada Percobaan 5 dengan menggunakan fitur histogram.

Import library

In [18]:
from pathlib import Path
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score
import cv2
import os
import random
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive

#Accessing my Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Image directories
train_dir = "/content/drive/MyDrive/UniLife/Semester5/ML/Jobsheet4/images/training/"
test_dir = "/content/drive/MyDrive/UniLife/Semester5/ML/Jobsheet4/images/test/"

Load Data

In [4]:
def load_dataset(img_dir):
    p = Path(img_dir)
    dirs = p.glob('*')

    img_list = []

    for dir in dirs:
        label = str(dir).split('/')[-1]
        for file in dir.glob('*.jpg'):
            img = mpimg.imread(file)

            if not img is None:
                img_list.append((img, label))

    return img_list

Standarisasi gambar

In [9]:
def standarized_input(image):
    # resize to w: 1100, h:600
    std_img = cv2.resize(image, (1100,600))
    return std_img

Encoding Label

In [10]:
def label_encoder(label):
    # Encode the label
    # day as 1; night as 0
    num_val = 0

    if(label == 'day'):
        num_val = 1
    else : 0

    return num_val

Pra pengolahan data

In [11]:
def preprocess(img_list):
    std_img_list = []

    for item in img_list:
        image = item[0]
        label = item[1]

        # Standarized the image
        std_img = standarized_input(image)

        # Create the label
        img_label = label_encoder(label)

        std_img_list.append((std_img, img_label))

    return std_img_list

Ekstraksi histrogram

In [12]:
def histogram_extract(image_path, bins=(8, 8, 8)):
    image = cv2.imread(image_path)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

Load data dan label

In [15]:
def load_data(dir):
    data = []
    labels = []

    for label in ["day", "night"]:
        path = os.path.join(dir, label)
        for img_file in os.listdir(path):
            img_path = os.path.join(path, img_file)
            hist = histogram_extract(img_path)
            data.append(hist)
            labels.append(0 if label == "day" else 1)

    return np.array(data), np.array(labels)

Load data training dan datda test

In [16]:
train_data, train_labels = load_data(train_dir)
test_data, test_labels = load_data(test_dir)

Klasifikasi

In [19]:
model = svm.SVC(kernel='linear')

model.fit(train_data, train_labels)

predictions = model.predict(test_data)

accuracy = accuracy_score(test_labels, predictions)
report = classification_report(test_labels, predictions, target_names=["Day", "Night"])

print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:")
print(report)

Accuracy: 97.50%
Classification Report:
              precision    recall  f1-score   support

         Day       0.99      0.96      0.97        80
       Night       0.96      0.99      0.98        80

    accuracy                           0.97       160
   macro avg       0.98      0.98      0.97       160
weighted avg       0.98      0.97      0.97       160

